# Robust Optimization Final Project Code

## Declare Variables and Input Information

In [12]:
using JuMP, Gurobi, CSV, DataFrames

const GRB_ENV = Gurobi.Env()

Gurobi.Env(Ptr{Nothing} @0x000000007e4512a0, false, 0)

### Update Every Time

In [13]:
scenario_name = "3x3"
w=12
h=12
batteryCapacity = 10000

10000

In [14]:
dir2 = "./";

# DR_input = dir2 * "Detection.csv";
# E_input = dir2 * "Adjacency.csv";
# T_input = dir2 * "Time.csv";
# robust_output = dir2 * "robust_output.csv";
# robust_path = dir2*"robust_path.csv";

# #read in the data files
# DR = CSV.read(DR_input, DataFrame, header=0);
# T = CSV.read(T_input, DataFrame, header=0);
# E = CSV.read(E_input, DataFrame, header=0);
# tri = CSV.read("100_triangles.csv", DataFrame, header=0);
# D_b = CSV.read(dir2* "Detection_buffer.csv", DataFrame, header=0);
# n = size(T)[1];

wh_string=string(w)*"_"*string(h)
arcs_file="arcs_"*wh_string*"_"*scenario_name*".csv";
tri_file=wh_string*"_triangle.csv"
# CS_file = wh_string*"_CS.csv"
# arcs= CSV.read(arcs_file, DataFrame, header=0);
# tri= CSV.read(tri_file, DataFrame, header=0);
# CS = CSV.read(CS_file, DataFrame, header=0);
# ins = CSV.read(wh_string*"_inflow.csv", DataFrame, header=0);
# outs = CSV.read(wh_string*"_outflow.csv", DataFrame, header=0);

tri= CSV.read("Buckner_tris_12_12.csv", DataFrame, header=0);
arcs= CSV.read("Buckner_arcs_12_12.csv", DataFrame, header=0);
ins = CSV.read("Buckner_ins_12_12.csv", DataFrame, header=0);
outs = CSV.read("Buckner_outs_12_12.csv", DataFrame, header=0);

In [15]:
D_lin = arcs[!,4];
T_lin = arcs[!,5];
E_lin = arcs[!,6];
# C=CS[!,1];

tris=[]
for t in eachrow(tri)
    new_t=[]
    for a in t
        if a!=0
            append!(new_t,a)
        end
    end
    push!(tris,new_t)
end
IN=size(ins,1)
inflow=[]
outflow=[]
for i in 1:IN
    new_in=[]
    new_out=[]
    for j in 1:10
        if ins[i,j]!=0
            append!(new_in,ins[i,j])
            append!(new_out,outs[i,j])
        end
    end
    push!(inflow,new_in)
    push!(outflow,new_out)
end        

In [16]:
#User Input and Initial Calculations
τ = 60*60*60; #number of seconds to traverse the graph
# S_max=7
# C_max=5
s = 1 #the starting node of the troops
f = 144 #the destination node of the troops
N=w*h*2;
A = 2312

2312

## Solve the Model

In [17]:
@time begin
    pathrob,objT,sTime = two_step_en(w,h,s,f,A,N,D_lin,T_lin,E_lin,tris,τ,inflow,outflow)
    end 

write_path(pathrob, A)

Set parameter TimeLimit to value 120
Set parameter MIPGap to value 0.01


LoadError: BoundsError: attempt to access 288-element Vector{Any} at index [289]

## Change OBJ Function

In [18]:
@time begin
    pathrob,objT,sTime = two_step_optimization(w,h,s,f,A,N,D_lin,T_lin,E_lin,tris,τ,inflow,outflow, batteryCapacity)
    end 

write_path(pathrob, A)

Set parameter TimeLimit to value 120
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 120
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6675 rows, 2312 columns and 28984 nonzeros
Model fingerprint: 0x54937824
Variable types: 0 continuous, 2312 integer (2312 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+05]
Found heuristic solution: objective 2900.0000000
Presolve removed 31 rows and 8 columns
Presolve time: 0.08s
Presolved: 6644 rows, 2304 columns, 26560 nonzeros
Variable types: 0 continuous, 2304 integer (2304 binary)

Root relaxation: objective 1.100000e+03, 122 iterations, 0.03 seconds (0.00 work units)

    

"output_3x3.csv"

In [8]:
function two_step_en(w,h,s,f,A,N,d,t,E,tris,t_all,inflow,outflow)
    m = Model(() -> Gurobi.Optimizer(GRB_ENV));
    MAXTIME=120
    set_optimizer_attributes(m, "TimeLimit" => MAXTIME, "MIPGap" => 1e-2, "OutputFlag" => 1);
    @variable(m, x[1:A], Bin)
    @variable(m, batteryLevel[1:A] >= 0) # battery level at each step
    @objective(m, Min, sum(d[i]*x[i] for i in 1:A) + (sum(E[i]*x[i] for i in 1:A)))
    ## (1/maximum(E))*
    @constraint(m, [i in 1:N], sum(x[k] for k in inflow[i])<=1)
    @constraint(m, [i in 1:N], sum(x[k] for k in outflow[i])<=1)
    @constraint(m, [i in 1:N], sum(x[k] for k in inflow[i] if i!=f && i!=s)-sum(x[k] for k in outflow[i] if i!=f && i!=s)==0)
    @constraint(m, sum(x[k] for k in inflow[s])-sum(x[k] for k in outflow[s])==-1)
    @constraint(m, sum(x[k] for k in inflow[f])-sum(x[k] for k in outflow[f])==1)
    @constraint(m, batteryLevel[1] == batteryCapacity) # initialize battery level

    # tri_tot=length(tris)
    # @constraint(m, [t in 1:tri_tot], sum(x[t] for t in tris[t])<=1)
    @constraint(m, sum(t[i]*x[i] for i=1:A) <= t_all)

    @constraint(m, [i in 2:A], batteryLevel[i] == sum(batteryLevel[j] for j in inflow[i]) - E[i] * x[i])  # update battery level at each step
    @constraint(m, [i in 1:A], batteryLevel[i] >= 0) # make sure it is never below 0
    @constraint(m, [i in 1:A], batteryLevel[i] <= batteryCapacity * x[i])
    # @constraint(m, sum(E[i]*x[i] for i=1:A) <= batteryCapacity) 

    optimize!(m)
    #time=MOI.get(m, MOI.SolveTime())
    #D_opt=objective_value(m)
    
    #@objective(m, Min, sum(E[i]*x[i] for i in 1:A))
    #@constraint(m, sum(d[i]*x[i] for i in 1:A)<=D_opt)
    #optimize!(m)
    optimalpath = value.(x)
    objective = objective_value(m)
    #time+=MOI.get(m, MOI.SolveTime())
    print(optimalpath)
    return optimalpath, objective, time
end

function write_path(opt_path, A)
    path=[]
    for i in 1:A
        if opt_path[i]==1
            append!(path,i)
        end
    end
    K=length(path)
    path_table=zeros(K,2)
    for i in 1:K
        path_table[i]=path[i]
    end
    path_output3 = DataFrame(path_table, :auto);
    CSV.write("output_"*scenario_name*".csv", path_output3); 
end

write_path (generic function with 1 method)

In [10]:
function two_step_optimization(w, h, s, f, A, N, d, t, E, tris, τ, inflow, outflow, batteryCapacity)
    m1 = Model(() -> Gurobi.Optimizer(GRB_ENV))
    MAXTIME = 120
    set_optimizer_attributes(m1, "TimeLimit" => MAXTIME, "MIPGap" => 1e-2, "OutputFlag" => 1)

    # Step 1: Minimize detection to find the least-detection path
    @variable(m1, x[1:A], Bin)
    @objective(m1, Min, sum(d[i] * x[i] for i in 1:A))

    @constraint(m1, [i in 1:N], sum(x[k] for k in inflow[i]) <= 1)
    @constraint(m1, [i in 1:N], sum(x[k] for k in outflow[i]) <= 1)
    @constraint(m1, [i in 1:N], 
        sum(x[k] for k in inflow[i] if i != f && i != s) - 
        sum(x[k] for k in outflow[i] if i != f && i != s) == 0)
    @constraint(m1, sum(x[k] for k in inflow[s]) - sum(x[k] for k in outflow[s]) == -1)
    @constraint(m1, sum(x[k] for k in inflow[f]) - sum(x[k] for k in outflow[f]) == 1)

    tri_tot = length(tris)
    @constraint(m1, [t in 1:tri_tot], sum(x[t] for t in tris[t]) <= 1)
    @constraint(m1, sum(t[i] * x[i] for i in 1:A) <= τ)

    optimize!(m1)

    # Extract the least-detection path
    optimal_path = value.(x)
    path_cost = objective_value(m1)

    println("Least detection path: ", optimal_path)
    println("Detection cost: ", path_cost)

    # Step 2: Optimize generator usage along the given path
    m2 = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attributes(m2, "TimeLimit" => MAXTIME, "MIPGap" => 1e-2, "OutputFlag" => 1)

    @variable(m2, batteryLevel[1:A] >= 0) # Battery level at each step
    @variable(m2, y[1:A], Bin) # Generator on/off decision

    @objective(m2, Min, sum(E[i] * optimal_path[i] * (1 - y[i]) for i in 1:A))

    @constraint(m2, batteryLevel[1] == batteryCapacity) # Initialize battery level
   #  @constraint(m2, [i in 2:A], 
   #     batteryLevel[i] == batteryLevel[i - 1] - E[i - 1] * optimal_path[i - 1] * (1 - y[i - 1]))
    @constraint(m2, [i in 1:A], batteryLevel[i] >= 0) # Ensure battery level is non-negative
    @constraint(m2, [i in 1:A], batteryLevel[i] <= batteryCapacity)
    @constraint(m2, sum(E[i] * optimal_path[i] * (1 - y[i]) for i in 1:A) <= batteryCapacity)

    optimize!(m2)

    generator_decision = value.(y)
    final_objective = objective_value(m2)

    println("Generator decisions: ", generator_decision)
    println("Final energy cost: ", final_objective)

    return optimal_path, generator_decision, path_cost, final_objective
end

function write_path(opt_path, A)
    path=[]
    for i in 1:A
        if opt_path[i]==1
            append!(path,i)
        end
    end
    K=length(path)
    path_table=zeros(K,2)
    for i in 1:K
        path_table[i]=path[i]
    end
    path_output3 = DataFrame(path_table, :auto);
    CSV.write("output_"*scenario_name*".csv", path_output3); 
end

write_path (generic function with 1 method)

In [46]:
write_path(pathrob, A)

print(optimalpath)

LoadError: UndefVarError: optimalpath not defined

## Optimization Functions

In [6]:
function two_step_en_v2(w, h, s, f, A, N, d, t, E, tris, τ, inflow, outflow, batteryCapacity)
    m_det = Model(() -> Gurobi.Optimizer(GRB_ENV))
    MAXTIME = 300
    set_optimizer_attributes(m_det, "TimeLimit" => MAXTIME, "MIPGap" => 1e-2, "OutputFlag" => 1)

    # Decision variables
    @variable(m_det, x[1:A], Bin)

    # Objective: Minimize total time
    @objective(m_det, Min, sum(d[i] * x[i] for i in 1:A))

    # Constraints
    @constraint(m_det, [i in 1:N], sum(x[k] for k in inflow[i]) <= 1)
    @constraint(m_det, [i in 1:N], sum(x[k] for k in outflow[i]) <= 1)
    @constraint(m_det, [i in 1:N], sum(x[k] for k in inflow[i] if i != f && i != s) -
                                   sum(x[k] for k in outflow[i] if i != f && i != s) == 0)
    @constraint(m_det, sum(x[k] for k in inflow[s]) - sum(x[k] for k in outflow[s]) == -1)
    @constraint(m_det, sum(x[k] for k in inflow[f]) - sum(x[k] for k in outflow[f]) == 1)
    tri_tot = length(tris)
    @constraint(m_det, [t in 1:tri_tot], sum(x[t] for t in tris[t]) <= 1)

    # Optimize the first step
    optimize!(m_det)
    D_opt = objective_value(m_det)
    optimalpath_det = value.(x)

    println("Step 1 - Optimal time path: $optimalpath_det with det $D_opt")
    # Step 2: Minimize energy with time constraint and influence of time-optimized path
    m_energy = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attributes(m_energy, "TimeLimit" => MAXTIME, "MIPGap" => 1e-2, "OutputFlag" => 1)

    # Decision variables
    @variable(m_energy, x[1:A], Bin)
    @variable(m_energy, batteryLevel[1:A] >= 0) # Battery level at each step

    # Parameter to weigh deviation penalty
    #  α = .1  # Adjust this parameter to control weight of time-path influence

    # Objective: Minimize energy with penalty for deviation from optimal time path
    # @objective(m_energy, Min, sum((1/maximum(E)) *E[i] * x[i] for i in 1:A) + α * sum((x[i] - .0001* optimalpath_det[i]) for i in 1:A))

    # Constraints (same as before)
    @constraint(m_energy, [i in 1:N], sum(x[k] for k in inflow[i]) <= 1)
    @constraint(m_energy, [i in 1:N], sum(x[k] for k in outflow[i]) <= 1)
    @constraint(m_energy, [i in 1:N], sum(x[k] for k in inflow[i] if i != f && i != s) -
                                    sum(x[k] for k in outflow[i] if i != f && i != s) == 0)
    @constraint(m_energy, sum(x[k] for k in inflow[s]) - sum(x[k] for k in outflow[s]) == -1)
    @constraint(m_energy, sum(x[k] for k in inflow[f]) - sum(x[k] for k in outflow[f]) == 1)
    @constraint(m_energy, sum(t[i] * x[i] for i in 1:A) <= D_opt + τ)  # Time constraint
    @constraint(m_energy, batteryLevel[1] == batteryCapacity) # Initial battery level
    @constraint(m_energy, [i in 2:A], batteryLevel[i] == batteryLevel[i - 1] - E[i - 1] * x[i - 1])  # Update battery level
    @constraint(m_energy, [i in 1:A], batteryLevel[i] >= 0) # Ensure battery is non-negative
    @constraint(m_energy, [i in 1:A], batteryLevel[i] <= batteryCapacity)
    @constraint(m_energy, sum(E[i] * x[i] for i in 1:A) <= batteryCapacity)

    # Optimize the second step
    optimize!(m_energy)
    pathrob = value.(x)
    objective_energy = objective_value(m_energy)

    return pathrob
end

two_step_en_v2 (generic function with 1 method)